In [1]:
import sys
import os

# Add the parent directory to the sys.path to avoid 'ModuleNotFoundError'
sys.path.append(os.path.abspath(os.path.join('..')))

import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["font.family"] = ['serif']

from src.helpers import load_json, load_pickle
from src.paths import paths
from src.config import MODEL_NAMES

## 1. Check if metrics have normal distribution

In [58]:
metrics_dict = load_json(paths.get('metric_fold_path'))
metrics_to_analyze = [
    'accuracy',
    'roc_auc',
    'f1_score',
    'precision',
    'recall',
    'specificity'
]

In [59]:
def check_normality(metrics_dict, metrics_to_analyze):
    results = []

    for model, metrics_list in metrics_dict.items():
        for metric in metrics_to_analyze:
            values = [metrics[metric] for metrics in metrics_list]
            stat, p_value = stats.shapiro(values)
            result = {
                'model': model,
                'metric': metric,
                'statistic': stat,
                'p_value': p_value,
                'normality': 'normal' if p_value > 0.05 else 'not normal'
            }
            results.append(result)
    
    results_df = pd.DataFrame(results)
    return results_df

normality_results_df = check_normality(metrics_dict, metrics_to_analyze)
normality_results_df

,model,metric,statistic,p_value,normality
0,catboost,accuracy,0.946522,0.712336,normal
1,catboost,roc_auc,0.838784,0.161588,normal
2,catboost,f1_score,0.980866,0.939200,normal
3,catboost,precision,0.940116,0.666758,normal
4,catboost,recall,0.802990,0.085693,normal
5,catboost,specificity,0.910757,0.472151,normal
6,xgboost,accuracy,0.977634,0.921584,normal
7,xgboost,roc_auc,0.800256,0.081425,normal
8,xgboost,f1_score,0.963147,0.829707,normal
9,xgboost,precision,0.946566,0.712645,normal


## 2. Statistical tests

### 2.1. Comparing Model Performance for normally distributed metrics

In [71]:
def extract_metric_scores(metrics_dict, metric_name):
    metric_scores = {}
    for model_name in MODEL_NAMES:
        metric_scores[model_name] = [score[metric_name] for score in metrics_dict[model_name]]
    return metric_scores

def prepare_data_for_anova(metric_scores, metric_name):
    data = pd.DataFrame(metric_scores)
    data['Fold'] = list(range(1, 6))  # Adding fold information
    data_long = pd.melt(data, id_vars=['Fold'], value_vars=data.columns[:-1], var_name='Model', value_name=metric_name)
    return data_long

def perform_repeated_measures_anova(data, metric_name):
    rm_anova = AnovaRM(data, metric_name, 'Fold', within=['Model']).fit()
    return rm_anova.summary()

def perform_tukey_hsd(data, metric_name):
    tukey = pairwise_tukeyhsd(endog=data[metric_name], groups=data['Model'], alpha=0.05)
    return tukey.summary()

def analyze_metric(metric_scores, metric_name):
    data = prepare_data_for_anova(metric_scores, metric_name)

    # Perform parametric tests
    anova_summary = perform_repeated_measures_anova(data, metric_name)
    print(anova_summary)

    tukey_summary = perform_tukey_hsd(data, metric_name)
    print(tukey_summary)

    return anova_summary

In [72]:
metrics_to_analyze = ['accuracy', 'roc_auc', 'f1_score', 'precision']

for metric_name in metrics_to_analyze:
    print(f"\nAnalyzing {metric_name.upper()}...\n")
    metric_scores = extract_metric_scores(metrics_dict, metric_name)
    analyze_metric(metric_scores, metric_name)


Analyzing ACCURACY...

               Anova
      F Value Num DF  Den DF Pr > F
-----------------------------------
Model  6.0849 5.0000 20.0000 0.0014

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1   group2 meandiff p-adj   lower  upper  reject
------------------------------------------------------
catboost    lgbm   0.0001    1.0 -0.0877 0.0878  False
catboost      lr  -0.0659 0.2238 -0.1537 0.0218  False
catboost      rf  -0.0403 0.7157  -0.128 0.0475  False
catboost     svm  -0.0624 0.2744 -0.1502 0.0253  False
catboost xgboost  -0.0147 0.9949 -0.1025  0.073  False
    lgbm      lr   -0.066 0.2229 -0.1537 0.0218  False
    lgbm      rf  -0.0403 0.7143 -0.1281 0.0474  False
    lgbm     svm  -0.0625 0.2734 -0.1502 0.0253  False
    lgbm xgboost  -0.0148 0.9947 -0.1026 0.0729  False
      lr      rf   0.0257 0.9417 -0.0621 0.1134  False
      lr     svm   0.0035    1.0 -0.0843 0.0913  False
      lr xgboost   0.0512 0.4824 -0.0366 0.1389  False
      rf     svm  -0.

### 2.2. Comparing Model Performance for not normally distributed metrics

In [81]:
from scipy.stats import friedmanchisquare

def perform_friedman_test(metrics_dict, metric_name):
    # Extract metric scores for all models
    metric_scores = extract_metric_scores(metrics_dict, metric_name)
    print(metric_scores)
    
    # Prepare the data in the format required for the Friedman test
    # Each model's metric scores should be passed as a separate argument
    scores = [metric_scores[model_name] for model_name in metric_scores]
    print(scores)
    
    # Perform the Friedman test
    stat, p_value = friedmanchisquare(*scores)
    
    # Print the result
    print(f"Friedman test result for {metric_name}:")
    print(f"Test Statistic: {stat}")
    print(f"P-Value: {p_value}")
    
    if p_value < 0.05:
        print(f"The differences in {metric_name} between models are statistically significant.")
    else:
        print(f"No significant differences in {metric_name} between models were found.")
    print('')


In [82]:
metrics_to_analyze = ['accuracy', 'roc_auc', 'f1_score', 'precision', 'recall', 'specificity']

for metric_name in metrics_to_analyze:
    perform_friedman_test(metrics_dict, metric_name)

{'catboost': [0.8545454545454545, 0.8909090909090909, 0.9090909090909091, 0.8703703703703703, 0.8703703703703703], 'xgboost': [0.7818181818181819, 0.9090909090909091, 0.9454545454545454, 0.8333333333333334, 0.8518518518518519], 'lgbm': [0.8, 0.9272727272727272, 0.9090909090909091, 0.8888888888888888, 0.8703703703703703], 'rf': [0.8, 0.8909090909090909, 0.8363636363636363, 0.8518518518518519, 0.8148148148148148], 'svm': [0.8, 0.8909090909090909, 0.8181818181818182, 0.7592592592592593, 0.8148148148148148], 'lr': [0.7636363636363637, 0.8727272727272727, 0.8181818181818182, 0.7962962962962963, 0.8148148148148148]}
[[0.8545454545454545, 0.8909090909090909, 0.9090909090909091, 0.8703703703703703, 0.8703703703703703], [0.7818181818181819, 0.9090909090909091, 0.9454545454545454, 0.8333333333333334, 0.8518518518518519], [0.8, 0.9272727272727272, 0.9090909090909091, 0.8888888888888888, 0.8703703703703703], [0.8, 0.8909090909090909, 0.8363636363636363, 0.8518518518518519, 0.8148148148148148], [0.

In [83]:
import numpy as np
from scipy.stats import rankdata
from scikit_posthocs import posthoc_nemenyi_friedman

def identify_different_models(metrics_dict, metric_name):
    # Extract metric scores for all models
    metric_scores = extract_metric_scores(metrics_dict, metric_name)
    
    # Prepare the data in the format required for the Friedman test
    scores = np.array([metric_scores[model_name] for model_name in metric_scores])

    # Perform the Friedman test
    stat, p_value = friedmanchisquare(*scores)
    
    print(f"Friedman test result for {metric_name}:")
    print(f"Test Statistic: {stat}")
    print(f"P-Value: {p_value}")
    
    if p_value < 0.05:
        print(f"The differences in {metric_name} between models are statistically significant.")
        
        # Perform the Nemenyi post-hoc test
        nemenyi_results = posthoc_nemenyi_friedman(scores.T)
        
        print("\nNemenyi post-hoc test results:")
        print(nemenyi_results)
        
        # Highlight significant differences
        significant_pairs = np.where(nemenyi_results < 0.05)
        for i in range(len(significant_pairs[0])):
            model1 = list(metric_scores.keys())[significant_pairs[0][i]]
            model2 = list(metric_scores.keys())[significant_pairs[1][i]]
            print(f"Significant difference between {model1} and {model2}")
        
    else:
        print(f"No significant differences in {metric_name} between models were found.")


In [84]:
metrics_to_analyze = ['accuracy', 'roc_auc', 'f1_score', 'precision', 'recall', 'specificity']

for metric_name in metrics_to_analyze:
    identify_different_models(metrics_dict, metric_name)

Friedman test result for accuracy:
Test Statistic: 16.187499999999996
P-Value: 0.00632865051737763
The differences in accuracy between models are statistically significant.

Nemenyi post-hoc test results:
          0         1         2         3         4         5
0  1.000000  0.900000  0.900000  0.727962  0.280498  0.059278
1  0.900000  1.000000  0.900000  0.900000  0.679147  0.280498
2  0.900000  0.900000  1.000000  0.532706  0.138788  0.021823
3  0.727962  0.900000  0.532706  1.000000  0.900000  0.679147
4  0.280498  0.679147  0.138788  0.900000  1.000000  0.900000
5  0.059278  0.280498  0.021823  0.679147  0.900000  1.000000
Significant difference between lgbm and lr
Significant difference between lr and lgbm
Friedman test result for roc_auc:
Test Statistic: 16.65714285714286
P-Value: 0.005198098170259067
The differences in roc_auc between models are statistically significant.

Nemenyi post-hoc test results:
          0         1         2         3         4         5
0  1.00000

### 2.3. Comparing the Impact of Features Using SHAP Values

In [7]:
shap_values = load_pickle(paths.get('shap_values_path'))

In [8]:
shap_values['catboost'][0].shape

(54, 20)

In [9]:
len(shap_values['catboost'][0])

54